# Adipocytes Tools

The **Adipocytes Tools** help to analyze fat cells in images from histological sections such as this one: [0178_x5_3.tif](http://dev.mri.cnrs.fr/attachments/190/0178_x5_3.tif). 

The source code in git-hub can be found [here](https://github.com/MontpellierRessourcesImagerie/imagej_macros_and_scripts/tree/master/volker/toolsets/adipocyte_tools).

In [19]:
image = read_image_from_url('http://dev.mri.cnrs.fr/attachments/190/0178_x5_3.tif')
imshow(image, width=400, height=300)

# Installation in FIJI/ImageJ

To install the tools, drag the link [Adipocyte_Tools.ijm](https://raw.githubusercontent.com/MontpellierRessourcesImagerie/imagej_macros_and_scripts/master/volker/toolsets/adipocyte_tools/Adipocyte_Tools.ijm) to the ImageJ launcher window and save it under _macros/toolsets_ in the ImageJ installation. In order to use the greyscale-watershed method (the **w**-button) you must have the [Watershed Algorithm](http://rsbweb.nih.gov/ij/plugins/watershed.html) installed. 

Select the "MRI Adipocytes Tools" toolset from the ``>>`` button of the ImageJ launcher.

![toolset](http://dev.mri.cnrs.fr/attachments/download/1691/toolset.png)

In [18]:
from beakerx import *

def read_image_from_url(url):
    import requests
    import tifffile as tiff
    import io
    resp = requests.get(url)
    return tiff.imread(io.BytesIO(resp.content))

def imshow(image, width=400, height=300):
    import IPython
    import ipywidgets as widgets
    
    displayImage = get_display_image(image, width=400, height=300)

    SID_image = widgets.Image(
        value=displayImage.data.tobytes(),
        format='jpg',
        width=400,
        height=300,
    )
    SID_slider = widgets.IntSlider(
        min=100,
        max=1900,
        continuous_update=False,
        value = 400,
        description = 'width',
        layout=Layout(width='400px')
    )

    SID_link = widgets.jslink((SID_image, 'width'), (SID_slider, 'value'))    

    SID_form = EasyForm("Image display")
    SID_form.addWidget('FloatSlider', SID_slider)
    SID_form.addWidget('ImageDisplay', SID_image)
    display(SID_form)
    
def get_display_image(img, width=800, height=800):
    import cv2
    import IPython
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret, width=width, height=height)
    return i
